In [201]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [347]:
df = pd.read_csv('spreadspoke_scores.csv')
df.dtypes

schedule_date           object
schedule_season          int64
schedule_week           object
team_home               object
team_away               object
stadium                 object
team_favorite_id        object
spread_favorite        float64
over_under_line         object
weather_detail          object
weather_temperature    float64
weather_wind_mph       float64
weather_humidity        object
score_home               int64
score_away               int64
stadium_neutral           bool
schedule_playoff          bool
game_id                 object
dtype: object

In [348]:
df = df.dropna(subset=['team_favorite_id', 'spread_favorite', 'over_under_line'])
df = df.drop(columns=['game_id','stadium_neutral','schedule_playoff', 'stadium','schedule_date'])

In [349]:
df['weather_humidity']=df['weather_humidity'].apply(pd.to_numeric, errors='coerce')
df['over_under_line']=df['over_under_line'].apply(pd.to_numeric, errors='coerce')
#df['schedule_date']=df['schedule_date'].apply(pd.to_datetime)

In [350]:
#cleaning team_fav_id to correctly match team names in instances where team moved city
df.loc[df.team_favorite_id =='SD','team_favorite_id'] = 'LAC'
df.loc[(df.team_favorite_id=='LAR') & (df.team_home!= 'Los Angeles Rams') & (df.team_away!= 'Los Angeles Rams') & (df.team_home!= 'St. Louis Rams') & (df.team_away!= 'St. Louis Rams'), 'team_favorite_id']='OAK'
df.loc[((df.team_home=='Baltimore Colts') | (df.team_away=='Baltimore Colts')) & (df.team_favorite_id=='BAL'), 'team_favorite_id'] = 'IND'

In [351]:
special_weather_df = df[(df.weather_detail.isna() == False) & (df.weather_detail != 'DOME')]

In [352]:
#convert playoff rounds into ints (started at 19 bc seasons '93 and '99 had 18 weeks)
df.loc[(df.schedule_week=='WildCard') | (df.schedule_week=='Wildcard'), 'schedule_week']=19
df.loc[(df.schedule_week=='Division'), 'schedule_week']=20
df.loc[(df.schedule_week=='Conference'), 'schedule_week']=21
df.loc[(df.schedule_week=='Superbowl') | (df.schedule_week=='SuperBowl'), 'schedule_week']=22

#convert into ints
df.schedule_week = df.schedule_week.astype('int64')

In [353]:
df['point_total'] = df['score_home'] + df['score_away']

In [354]:
df['over_under_result'] = np.where(df['point_total']>df['over_under_line'], 'over', 'under')
df['over_under_result'] = np.where(df['point_total']==df['over_under_line'], 'push', df['over_under_result'])

In [355]:
df['score_difference'] = df['score_home'] - df['score_away']

In [356]:
teams_dict = {'Baltimore Ravens': 'BAL', 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR', 'Green Bay Packers': 'GB','St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK','Detroit Lions': 'DET','Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR','Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU', 'Tennessee Titans': 'TEN', 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR', 'Baltimore Colts': 'IND', 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI', 'San Diego Chargers': 'LAC', 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS', 'Miami Dolphins': 'MIA', 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN', 'Minnesota Vikings': 'MIN', 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO', 'Pittsburgh Steelers': 'PIT','Kansas City Chiefs': 'KC', 
 'Denver Broncos': 'DEN', 'New York Giants': 'NYG', 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE', 'Philadelphia Eagles': 'PHI', 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI', 'Chicago Bears': 'CHI', 'New England Patriots': 'NE',
 'San Francisco 49ers': 'SF','Tampa Bay Buccaneers': 'TB','Seattle Seahawks': 'SEA'}
teams_dict

{'Baltimore Ravens': 'BAL',
 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR',
 'Green Bay Packers': 'GB',
 'St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK',
 'Detroit Lions': 'DET',
 'Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR',
 'Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU',
 'Tennessee Titans': 'TEN',
 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR',
 'Baltimore Colts': 'IND',
 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI',
 'San Diego Chargers': 'LAC',
 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS',
 'Miami Dolphins': 'MIA',
 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN',
 'Minnesota Vikings': 'MIN',
 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO',
 'Pittsburgh Steelers': 'PIT',
 'Kansas City Chiefs': 'KC',
 'Denver Broncos': 'DEN',
 'New York Giants': 'NYG',
 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE',
 'Philadelphia Eagles': 'PHI',
 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI',
 'Chicago 

In [357]:
#create home and away id columns
df['home_id'] = ''
df['away_id'] = ''
df['home_id'] = df.team_home.map(teams_dict)
df['away_id'] = df.team_away.map(teams_dict)


In [358]:
df['home_favorite'] = 0
df.loc[df.home_id == df.team_favorite_id,'home_favorite'] = 1
df

,schedule_season,schedule_week,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,weather_detail,weather_temperature,weather_wind_mph,weather_humidity,score_home,score_away,point_total,over_under_result,score_difference,home_id,away_id,home_favorite
350,1967,22,Green Bay Packers,Oakland Raiders,GB,-13.5,43.0,60 degrees- humidity 74%- wind 12 mph,60.0,12.0,74.0,33,14,47,over,19,GB,OAK,1
538,1968,22,Baltimore Colts,New York Jets,IND,-18.0,40.0,66 degrees- humidity 80%- wind 12 mph,66.0,12.0,80.0,7,16,23,under,-9,IND,NYJ,1
727,1969,22,Kansas City Chiefs,Minnesota Vikings,MIN,-12.0,39.0,55 degrees- humidity 84%- wind 14 mph,55.0,14.0,84.0,23,7,30,under,16,KC,MIN,0
916,1970,22,Baltimore Colts,Dallas Cowboys,IND,-2.5,36.0,59 degrees- humidity 60%- wind 11 mph,59.0,11.0,60.0,16,13,29,under,3,IND,DAL,1
1105,1971,22,Dallas Cowboys,Miami Dolphins,DAL,-6.0,34.0,34 degrees- humidity 40%- wind 18 mph- wind c...,34.0,18.0,40.0,24,3,27,under,21,DAL,MIA,1
1294,1972,22,Miami Dolphins,Washington Redskins,MIA,-1.0,33.0,64 degrees- humidity 53%- wind 7 mph,64.0,7.0,53.0,14,7,21,under,7,MIA,WAS,1
1483,1973,22,Miami Dolphins,Minnesota Vikings,MIA,-6.5,33.0,47 degrees- humidity 81%- wind 7 mph,47.0,7.0,81.0,24,7,31,under,17,MIA,MIN,1
1672,1974,22,Minnesota Vikings,Pittsburgh Steelers,PIT,-3.0,33.0,51 degrees- humidity 72%- wind 17 mph,51.0,17.0,72.0,6,16,22,under,-10,MIN,PIT,0
1861,1975,22,Dallas Cowboys,Pittsburgh Steelers,PIT,-7.0,36.0,49 degrees- humidity 44%- wind 18 mph,49.0,18.0,44.0,17,21,38,over,-4,DAL,PIT,0
2064,1976,22,Minnesota Vikings,Oakland Raiders,OAK,-4.0,38.0,52 degrees- humidity 59%- wind 6 mph,52.0,6.0,59.0,14,32,46,over,-18,MIN,OAK,0


In [359]:
#determine whether favorite covered the spread, or push if even
df['favorite_covered'] = np.where((df.home_favorite==1) & (np.abs(df.score_difference) > np.abs(df.spread_favorite)),1,0)
df['favorite_covered'] = np.where((np.abs(df.score_difference) == np.abs(df.spread_favorite)),'2',df.home_favorite)


In [360]:
#which team won the game outright (no spread)
df['winning_team'] = ''
df['winning_team'] = np.where(df.score_home > df.score_away, df.home_id, df.away_id)
df['winning_team'] = np.where(df.score_home == df.score_away, 'tie', df.winning_team)

df['losing_team'] = ''
df['losing_team'] = np.where(df.score_home < df.score_away, df.home_id, df.away_id)
df['losing_team'] = np.where(df.score_home == df.score_away, 'tie', df.losing_team)

In [361]:
df['home_wins'] = 0
df['home_losses'] = 0
df['home_ties'] = 0
df['away_wins'] = 0
df['away_losses'] = 0
df['away_ties'] = 0

In [345]:
sum((df['schedule_season'] == 2017) & (df['schedule_week'] < 5) & (df['winning_team'] == 'NE'))

2

In [414]:
def get_record(df):
    home_wins = []
    home_losses = []
    home_ties = []
    away_wins = []
    away_losses = []
    away_ties = []
    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        home_id = row.home_id
        away_id = row.away_id
        
        hw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == home_id))
        home_wins.append(hw)
        hl = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == home_id))
        home_losses.append(hl)
        ht = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['home_id'] == home_id))
        home_ties.append(ht)
        
        aw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == away_id))
        away_wins.append(aw) 
        al = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == away_id))
        away_losses.append(al)
        at = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['away_id'] == away_id))
        away_ties.append(at)
        
    return home_wins, home_losses, home_ties, away_wins, away_losses, away_ties
        
home_wins, home_losses, home_ties, away_wins, away_losses, away_ties = get_record(df)

In [280]:
df_filter = df.filter(items=['schedule_season', 'schedule_week', 'winning_team', 'home_id','home_wins'])

In [415]:
df['home_wins'] = home_wins
df['home_losses'] = home_losses
df['home_ties'] = home_ties
df['away_wins'] = away_wins
df['away_losses'] = away_losses
df['away_ties'] = away_ties


df[(df.schedule_season==2017) & ((df.home_id == 'NYG') | (df.away_id == 'NYG'))]

,schedule_season,schedule_week,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,weather_detail,weather_temperature,weather_wind_mph,...,home_favorite,favorite_covered,winning_team,losing_team,home_wins,home_losses,home_ties,away_wins,away_losses,away_ties
11882,2017,1,Dallas Cowboys,New York Giants,DAL,-3.5,47.5,DOME,72.0,0.0,...,1,1,DAL,NYG,0,0,0,0,0,0
11907,2017,2,New York Giants,Detroit Lions,NYG,-3.5,43.5,NaN,72.0,7.0,...,1,1,DET,NYG,0,1,0,1,0,0
11920,2017,3,Philadelphia Eagles,New York Giants,PHI,-6.0,43.0,NaN,83.0,7.0,...,1,1,PHI,NYG,1,1,0,0,2,0
11938,2017,4,Tampa Bay Buccaneers,New York Giants,TB,-3.0,44.5,Rain,82.0,11.0,...,1,1,TB,NYG,1,1,0,0,3,0
11949,2017,5,New York Giants,Los Angeles Chargers,NYG,-3.5,44.5,NaN,75.0,10.0,...,1,1,LAC,NYG,0,4,0,0,4,0
11958,2017,6,Denver Broncos,New York Giants,DEN,-12.0,39.0,NaN,60.0,4.0,...,1,1,NYG,DEN,3,1,0,0,5,0
11979,2017,7,New York Giants,Seattle Seahawks,SEA,-3.5,39.0,NaN,74.0,7.0,...,0,0,SEA,NYG,1,5,0,3,2,0
12003,2017,9,New York Giants,Los Angeles Rams,LAR,-5.5,42.5,NaN,56.0,6.0,...,0,0,LAR,NYG,1,6,0,5,2,0
12018,2017,10,San Francisco 49ers,New York Giants,NYG,-3.0,41.5,NaN,68.0,5.0,...,0,0,SF,NYG,0,9,0,1,7,0
12034,2017,11,New York Giants,Kansas City Chiefs,KC,-10.0,45.5,NaN,49.0,18.0,...,0,0,NYG,KC,1,8,0,6,3,0


In [416]:
np.sum(df.away_wins) + np.sum(df.home_wins) 

77873

In [421]:
df[df['home_ties']>0]

,schedule_season,schedule_week,team_home,team_away,team_favorite_id,spread_favorite,over_under_line,weather_detail,weather_temperature,weather_wind_mph,...,home_favorite,favorite_covered,winning_team,losing_team,home_wins,home_losses,home_ties,away_wins,away_losses,away_ties
2857,1980,9,Tampa Bay Buccaneers,New York Giants,TB,-7.0,39.0,NaN,67.0,8.0,...,1,1,TB,NYG,3,4,1,1,7,0
2872,1980,10,Tampa Bay Buccaneers,Pittsburgh Steelers,PIT,-7.0,43.0,NaN,74.0,6.0,...,0,0,PIT,TB,4,4,1,5,4,0
2900,1980,12,Tampa Bay Buccaneers,Detroit Lions,TB,-3.0,39.0,NaN,69.0,9.0,...,1,1,DET,TB,4,6,1,6,5,0
2927,1980,14,Tampa Bay Buccaneers,Minnesota Vikings,TB,-2.0,39.0,NaN,64.0,7.0,...,1,1,MIN,TB,5,7,1,7,6,0
2945,1980,16,Tampa Bay Buccaneers,Chicago Bears,CHI,-1.0,35.0,NaN,59.0,8.0,...,0,2,CHI,TB,5,9,1,6,9,0
3058,1981,7,Miami Dolphins,Washington Redskins,MIA,-7.0,40.0,NaN,76.0,7.0,...,1,1,MIA,WAS,4,1,1,1,5,0
3083,1981,9,Miami Dolphins,Baltimore Colts,MIA,-7.0,48.0,NaN,78.0,22.0,...,1,1,MIA,IND,5,2,1,1,7,0
3112,1981,11,Miami Dolphins,Oakland Raiders,MIA,-5.0,39.0,NaN,69.0,8.0,...,1,1,OAK,MIA,7,2,1,4,6,0
3148,1981,13,Miami Dolphins,Philadelphia Eagles,PHI,-3.0,45.0,NaN,73.0,8.0,...,0,2,MIA,PHI,7,4,1,9,3,0
3155,1981,14,Miami Dolphins,New England Patriots,MIA,-6.5,44.0,NaN,60.0,13.0,...,1,1,MIA,NE,8,4,1,2,11,0
